In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [0]:
training_set_features = r"/content/drive/My Drive/LeadingIndiaAI/training_set_features.csv"
X = pd.read_csv(training_set_features,index_col="respondent_id")

training_set_labels = r"/content/drive/My Drive/LeadingIndiaAI/training_set_labels.csv"
y = pd.read_csv(training_set_labels,index_col="respondent_id")

In [0]:
print("features_df.shape", X.shape)
X.head()

features_df.shape (26707, 35)


,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [0]:
X.dtypes

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                             object
income_poverty           

In [0]:
print("labels_df.shape", y.shape)
y.head()

labels_df.shape (26707, 2)


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [0]:
test_set_features = r"/content/drive/My Drive/LeadingIndiaAI/test_set_features.csv"
test_file = pd.read_csv(test_set_features,index_col="respondent_id")
test_file.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,1.0,1.0,18 - 34 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,4.0,2.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,4.0,4.0,4.0,2.0,35 - 44 Years,12 Years,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


# **CatBoost Model (no need of one-hot-encoding....just load the data)**

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33)

In [0]:
!pip install catboost

     |████████████████████████████████| 64.8MB 63kB/s 


In [0]:
X = X.fillna(-999)
pd.isnull(X).any()

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=-999,strategy='most_frequent')
imputer=imputer.fit(X)
X_new=imputer.transform(X)
print(X_new)

[[1.0 0.0 0.0 ... 0.0 'fcxhlnwr' 'xtkaffoo']
 [3.0 2.0 0.0 ... 0.0 'pxcmvdjn' 'xgwztkwe']
 [1.0 1.0 0.0 ... 0.0 'rucpziij' 'xtkaffoo']
 ...
 [2.0 2.0 0.0 ... 0.0 'fcxhlnwr' 'xtkaffoo']
 [1.0 1.0 0.0 ... 0.0 'fcxhlnwr' 'haliazsg']
 [0.0 0.0 0.0 ... 0.0 'fcxhlnwr' 'xtkaffoo']]


In [0]:
test_file_no_na = test_file.fillna(-999)
pd.isnull(test_file_no_na).any()

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=-999,strategy='most_frequent')
imputer=imputer.fit(test_file_no_na)
test_file_new=imputer.transform(test_file_no_na)
print(test_file_new)

[[2.0 2.0 0.0 ... 0.0 'atmlpfrs' 'hfxkjkmi']
 [1.0 1.0 0.0 ... 0.0 'atmlpfrs' 'xqwwgdyp']
 [2.0 2.0 0.0 ... 0.0 'nduyfdeo' 'pvmttkik']
 ...
 [0.0 1.0 0.0 ... 0.0 'fcxhlnwr' 'xtkaffoo']
 [3.0 1.0 0.0 ... 0.0 'fcxhlnwr' 'xtkaffoo']
 [2.0 1.0 0.0 ... 0.0 'fcxhlnwr' 'xtkaffoo']]


In [0]:
y_h1n1=y['seasonal_vaccine']

In [0]:
from catboost import CatBoostClassifier
categorical_features_indices = np.where(X.dtypes != np.float)[0]
#model=CatBoostClassifier(loss_function='Logloss',cat_features=(['age_group','education','race','sex', 'income_poverty', 'marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa','employment_industry','employment_occupation']))
model=CatBoostClassifier(loss_function='Logloss',cat_features=categorical_features_indices)
#categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(X_new,y_h1n1)
#model.score(X_test,y_test)
                                                          #loss_function='CrossEntropy'
p=model.predict(test_file_new,prediction_type='Probability')  #prediction_type='Probability'

pp=model.predict_proba(test_file_new)

Learning rate set to 0.041889
0:	learn: 0.6758609	total: 62.5ms	remaining: 1m 2s
1:	learn: 0.6600181	total: 118ms	remaining: 58.9s
2:	learn: 0.6459327	total: 185ms	remaining: 1m 1s
3:	learn: 0.6331269	total: 255ms	remaining: 1m 3s
4:	learn: 0.6211483	total: 319ms	remaining: 1m 3s
5:	learn: 0.6102151	total: 375ms	remaining: 1m 2s
6:	learn: 0.6005067	total: 435ms	remaining: 1m 1s
7:	learn: 0.5913486	total: 492ms	remaining: 1m 1s
8:	learn: 0.5829926	total: 555ms	remaining: 1m 1s
9:	learn: 0.5752523	total: 622ms	remaining: 1m 1s
10:	learn: 0.5685852	total: 671ms	remaining: 1m
11:	learn: 0.5618584	total: 729ms	remaining: 1m
12:	learn: 0.5559391	total: 778ms	remaining: 59.1s
13:	learn: 0.5508340	total: 847ms	remaining: 59.6s
14:	learn: 0.5465989	total: 910ms	remaining: 59.8s
15:	learn: 0.5421004	total: 983ms	remaining: 1m
16:	learn: 0.5378657	total: 1.05s	remaining: 1m
17:	learn: 0.5339754	total: 1.1s	remaining: 59.9s
18:	learn: 0.5306303	total: 1.17s	remaining: 1m
19:	learn: 0.5277888	total

In [0]:
print(p)

[[0.81433726 0.18566274]
 [0.97366791 0.02633209]
 [0.19409361 0.80590639]
 ...
 [0.81520875 0.18479125]
 [0.68490313 0.31509687]
 [0.41147785 0.58852215]]


In [0]:
print(pp)

[[0.81433726 0.18566274]
 [0.97366791 0.02633209]
 [0.19409361 0.80590639]
 ...
 [0.81520875 0.18479125]
 [0.68490313 0.31509687]
 [0.41147785 0.58852215]]


In [0]:
np.savetxt('subm_seas_catboost(logloss,class)_imputed.csv',pp,delimiter=",")

# **Data Preprocessing**

In [0]:
for cat in ['age_group','education','race','sex', 'income_poverty', 'marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa','employment_industry','employment_occupation']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, X[cat].unique().size))

Number of levels in category 'age_group':  5.00 
Number of levels in category 'education':  5.00 
Number of levels in category 'race':  4.00 
Number of levels in category 'sex':  2.00 
Number of levels in category 'income_poverty':  4.00 
Number of levels in category 'marital_status':  3.00 
Number of levels in category 'rent_or_own':  3.00 
Number of levels in category 'employment_status':  4.00 
Number of levels in category 'hhs_geo_region':  10.00 
Number of levels in category 'census_msa':  3.00 
Number of levels in category 'employment_industry':  22.00 
Number of levels in category 'employment_occupation':  24.00 


In [0]:
for cat in ['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','census_msa']:
    print("Levels for catgeory '{0}': {1}".format(cat, X[cat].unique()))

Levels for catgeory 'age_group': ['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']
Levels for catgeory 'education': ['< 12 Years' '12 Years' 'College Graduate' 'Some College' nan]
Levels for catgeory 'race': ['White' 'Black' 'Other or Multiple' 'Hispanic']
Levels for catgeory 'sex': ['Female' 'Male']
Levels for catgeory 'income_poverty': ['Below Poverty' '<= $75,000, Above Poverty' '> $75,000' nan]
Levels for catgeory 'marital_status': ['Not Married' 'Married' nan]
Levels for catgeory 'rent_or_own': ['Own' 'Rent' nan]
Levels for catgeory 'employment_status': ['Not in Labor Force' 'Employed' 'Unemployed' nan]
Levels for catgeory 'census_msa': ['Non-MSA' 'MSA, Not Principle  City' 'MSA, Principle City']


In [0]:
for cat in ['employment_industry','employment_occupation','hhs_geo_region']:
    print("Levels for catgeory '{0}': {1}".format(cat, X[cat].unique()))

Levels for catgeory 'employment_industry': [nan 'pxcmvdjn' 'rucpziij' 'wxleyezf' 'saaquncn' 'xicduogh' 'ldnlellj'
 'wlfvacwt' 'nduyfdeo' 'fcxhlnwr' 'vjjrobsf' 'arjwrbjb' 'atmlpfrs'
 'msuufmds' 'xqicxuve' 'phxvnwax' 'dotnnunm' 'mfikgejo' 'cfqqtusy'
 'mcubkhph' 'haxffmxo' 'qnlwzans']
Levels for catgeory 'employment_occupation': [nan 'xgwztkwe' 'xtkaffoo' 'emcorrxb' 'vlluhbov' 'xqwwgdyp' 'ccgxvspp'
 'qxajmpny' 'kldqjyjy' 'mxkfnird' 'hfxkjkmi' 'bxpfxfdn' 'ukymxvdu'
 'cmhcxjea' 'haliazsg' 'dlvbwzss' 'xzmlyyjv' 'oijqvulv' 'rcertsgn'
 'tfqavkke' 'hodpvpew' 'uqqtjvyb' 'pvmttkik' 'dcjcmpih']
Levels for catgeory 'hhs_geo_region': ['oxchjgsf' 'bhuqouqj' 'qufhixun' 'lrircsnp' 'atmpeygn' 'lzgpxyit'
 'fpwskwrf' 'mlyzmhmf' 'dqpwygqj' 'kbazzjca']


In [0]:
X['employment_industry'] = X['employment_industry'].map({'pxcmvdjn':0,'rucpziij':1,'wxleyezf':2,'saaquncn':3,'xicduogh':4, 'ldnlellj':5,
 'wlfvacwt':6, 'nduyfdeo':7, 'fcxhlnwr':8, 'vjjrobsf':9, 'arjwrbjb':10, 'atmlpfrs':11,
 'msuufmds':12, 'xqicxuve':13, 'phxvnwax':14, 'dotnnunm':15, 'mfikgejo':16, 'cfqqtusy':17,
 'mcubkhph':18, 'haxffmxo':19, 'qnlwzans':20})

X['employment_occupation'] = X['employment_occupation'].map({'xgwztkwe':0,'xtkaffoo':1,'emcorrxb':2, 'vlluhbov':3, 'xqwwgdyp':4, 'ccgxvspp':5,
 'qxajmpny':6, 'kldqjyjy':7, 'mxkfnird':8, 'hfxkjkmi':9, 'bxpfxfdn':10, 'ukymxvdu':11,
 'cmhcxjea':12, 'haliazsg':13, 'dlvbwzss':14, 'xzmlyyjv':15, 'oijqvulv':16, 'rcertsgn':17,
 'tfqavkke':18, 'hodpvpew':19, 'uqqtjvyb':20, 'pvmttkik':21, 'dcjcmpih':22})

X['hhs_geo_region'] = X['hhs_geo_region'].map({'oxchjgsf':0,'bhuqouqj':1,'qufhixun':2, 'lrircsnp':3, 'atmpeygn':4, 'lzgpxyit':5,
 'fpwskwrf':6, 'mlyzmhmf':7, 'dqpwygqj':8, 'kbazzjca':9})

In [0]:
X['age_group'] = X['age_group'].map({'18 - 34 Years':0,'35 - 44 Years':1,'45 - 54 Years':2,'55 - 64 Years':3,'65+ Years':4})
X['education'] = X['education'].map({'< 12 Years':0,'12 Years':1,'College Graduate':2,'Some College':3})
X['race'] = X['race'].map({'White':0,'Black':1,'Hispanic':2,'Other or Multiple':3})
X['sex'] = X['sex'].map({'male':0,'12 Years':1,})
X['income_poverty'] = X['income_poverty'].map({'Below Poverty':0,'<= $75,000, Above Poverty':1,'> $75,000':2})
X['marital_status'] = X['marital_status'].map({'Not Married':0,'Married':1})
X['rent_or_own'] = X['rent_or_own'].map({'Own':0,'Rent':1})
X['employment_status'] = X['employment_status'].map({'Not in Labor Force':0,'Unemployed':1,'Employed':2})
X['census_msa'] = X['census_msa'].map({'Non-MSA':0,'MSA, Not Principle  City':1,'MSA, Principle City':2})

In [0]:
print(X['census_msa'])

respondent_id
0        0
1        1
2        1
3        2
4        1
        ..
26702    0
26703    2
26704    1
26705    0
26706    2
Name: census_msa, Length: 26707, dtype: int64


In [0]:
print(X['employment_industry'])
print(X['employment_occupation'])
print(X['hhs_geo_region'])

respondent_id
0        NaN
1        0.0
2        1.0
3        NaN
4        2.0
        ... 
26702    NaN
26703    8.0
26704    NaN
26705    8.0
26706    NaN
Name: employment_industry, Length: 26707, dtype: float64
respondent_id
0         NaN
1         0.0
2         1.0
3         NaN
4         2.0
         ... 
26702     NaN
26703    12.0
26704     NaN
26705    13.0
26706     NaN
Name: employment_occupation, Length: 26707, dtype: float64
respondent_id
0        0
1        1
2        2
3        3
4        2
        ..
26702    2
26703    5
26704    5
26705    3
26706    7
Name: hhs_geo_region, Length: 26707, dtype: int64


In [0]:
X = X.fillna(-999)
pd.isnull(X).any()

h1n1_concern                   False
h1n1_knowledge                 False
behavioral_antiviral_meds      False
behavioral_avoidance           False
behavioral_face_mask           False
behavioral_wash_hands          False
behavioral_large_gatherings    False
behavioral_outside_home        False
behavioral_touch_face          False
doctor_recc_h1n1               False
doctor_recc_seasonal           False
chronic_med_condition          False
child_under_6_months           False
health_worker                  False
health_insurance               False
opinion_h1n1_vacc_effective    False
opinion_h1n1_risk              False
opinion_h1n1_sick_from_vacc    False
opinion_seas_vacc_effective    False
opinion_seas_risk              False
opinion_seas_sick_from_vacc    False
age_group                      False
education                      False
race                           False
sex                            False
income_poverty                 False
marital_status                 False
r

In [0]:
test_set_features = r"/content/drive/My Drive/LeadingIndiaAI/test_set_features.csv"
test_file = pd.read_csv(test_set_features,index_col="respondent_id")
test_file.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,1.0,1.0,18 - 34 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,4.0,2.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,4.0,4.0,4.0,2.0,35 - 44 Years,12 Years,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [0]:
test_file['employment_industry'] = test_file['employment_industry'].map({'pxcmvdjn':0,'rucpziij':1,'wxleyezf':2,'saaquncn':3,'xicduogh':4, 'ldnlellj':5,
 'wlfvacwt':6, 'nduyfdeo':7, 'fcxhlnwr':8, 'vjjrobsf':9, 'arjwrbjb':10, 'atmlpfrs':11,
 'msuufmds':12, 'xqicxuve':13, 'phxvnwax':14, 'dotnnunm':15, 'mfikgejo':16, 'cfqqtusy':17,
 'mcubkhph':18, 'haxffmxo':19, 'qnlwzans':20})

test_file['employment_occupation'] = test_file['employment_occupation'].map({'xgwztkwe':0,'xtkaffoo':1,'emcorrxb':2, 'vlluhbov':3, 'xqwwgdyp':4, 'ccgxvspp':5,
 'qxajmpny':6, 'kldqjyjy':7, 'mxkfnird':8, 'hfxkjkmi':9, 'bxpfxfdn':10, 'ukymxvdu':11,
 'cmhcxjea':12, 'haliazsg':13, 'dlvbwzss':14, 'xzmlyyjv':15, 'oijqvulv':16, 'rcertsgn':17,
 'tfqavkke':18, 'hodpvpew':19, 'uqqtjvyb':20, 'pvmttkik':21, 'dcjcmpih':22})

test_file['hhs_geo_region'] = test_file['hhs_geo_region'].map({'oxchjgsf':0,'bhuqouqj':1,'qufhixun':2, 'lrircsnp':3, 'atmpeygn':4, 'lzgpxyit':5,
 'fpwskwrf':6, 'mlyzmhmf':7, 'dqpwygqj':8, 'kbazzjca':9})

In [0]:
test_file['age_group'] = test_file['age_group'].map({'18 - 34 Years':0,'35 - 44 Years':1,'45 - 54 Years':2,'55 - 64 Years':3,'65+ Years':4})
test_file['education'] = test_file['education'].map({'< 12 Years':0,'12 Years':1,'College Graduate':2,'Some College':3})
test_file['race'] = test_file['race'].map({'White':0,'Black':1,'Hispanic':2,'Other or Multiple':3})
test_file['sex'] = test_file['sex'].map({'male':0,'12 Years':1,})
test_file['income_poverty'] = test_file['income_poverty'].map({'Below Poverty':0,'<= $75,000, Above Poverty':1,'> $75,000':2})
test_file['marital_status'] = test_file['marital_status'].map({'Not Married':0,'Married':1})
test_file['rent_or_own'] = test_file['rent_or_own'].map({'Own':0,'Rent':1})
test_file['employment_status'] = test_file['employment_status'].map({'Not in Labor Force':0,'Unemployed':1,'Employed':2})
test_file['census_msa'] = test_file['census_msa'].map({'Non-MSA':0,'MSA, Not Principle  City':1,'MSA, Principle City':2})

In [0]:
test_file_new = test_file.fillna(-999)
pd.isnull(test_file_new).any()

h1n1_concern                   False
h1n1_knowledge                 False
behavioral_antiviral_meds      False
behavioral_avoidance           False
behavioral_face_mask           False
behavioral_wash_hands          False
behavioral_large_gatherings    False
behavioral_outside_home        False
behavioral_touch_face          False
doctor_recc_h1n1               False
doctor_recc_seasonal           False
chronic_med_condition          False
child_under_6_months           False
health_worker                  False
health_insurance               False
opinion_h1n1_vacc_effective    False
opinion_h1n1_risk              False
opinion_h1n1_sick_from_vacc    False
opinion_seas_vacc_effective    False
opinion_seas_risk              False
opinion_seas_sick_from_vacc    False
age_group                      False
education                      False
race                           False
sex                            False
income_poverty                 False
marital_status                 False
r

In [0]:
y_h1n1=y['h1n1_vaccine']
print(y_h1n1)

respondent_id
0        0
1        0
2        0
3        0
4        0
        ..
26702    0
26703    0
26704    0
26705    0
26706    0
Name: h1n1_vaccine, Length: 26707, dtype: int64


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_h1n1,
    test_size=0.33)

# **Ensemble Techniques**

Stacking

In [0]:
from sklearn.model_selection import StratifiedKFold
def Stacking(model,train,y,test,n_fold):
   folds=StratifiedKFold(n_splits=n_fold,random_state=1)
   test_pred=np.empty((test.shape[0],1),float)
   train_pred=np.empty((0,1),float)
   for train_indices,val_indices in folds.split(train,y.values):
      X_train,X_test=train.iloc[train_indices],train.iloc[val_indices]
      y_train,y_test=y.iloc[train_indices],y.iloc[val_indices]

      model.fit(X=X_train,y=y_train)
      train_pred=np.append(train_pred,model.predict(X_test))
      test_pred=np.append(test_pred,model.predict(y_test))
   return (test_pred.reshape(-1,1),train_pred)

In [0]:
from sklearn import tree
model1 = tree.DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10,train=X,test=test_file_new,y=y)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)


from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier()

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=X,test=test_file_new,y=y)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)



df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

from sklearn.linear_model import LinearRegression
model = LinearRegression(random_state=1)
model.fit(df)
model.score(df_test, test_file_new)

Bagging Classifier

In [0]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(X, y_seas)
#model.score(X_test,y_test)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=1,
   

In [0]:
pp1=model.predict_proba(test_file_new)
print(pp1)

[[0.9 0.1]
 [1.  0. ]
 [0.8 0.2]
 ...
 [0.8 0.2]
 [0.6 0.4]
 [0.6 0.4]]


In [0]:
np.savetxt('subm_seas_baggingclassifier.csv',pp1,delimiter=",")

AdaBoost Classifier

In [0]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(X, y_seas)
#model.score(X_test,y_test)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=1)

In [0]:
pp2=model.predict_proba(test_file_new)
print(pp2)

[[0.50367102 0.49632898]
 [0.51542929 0.48457071]
 [0.49641664 0.50358336]
 ...
 [0.50425656 0.49574344]
 [0.50240134 0.49759866]
 [0.50042716 0.49957284]]


In [0]:
np.savetxt('subm_seas_adaboostclass.csv',pp2,delimiter=",")

GradientBoosting Classifier

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(X, y_seas)
#model.score(X_test,y_test)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
pp3=model.predict_proba(test_file_new)
print(pp3)

[[0.62654741 0.37345259]
 [0.73166931 0.26833069]
 [0.37913463 0.62086537]
 ...
 [0.61684172 0.38315828]
 [0.56657547 0.43342453]
 [0.3823093  0.6176907 ]]


In [0]:
np.savetxt('subm_seas_gradientboostclass.csv',pp3,delimiter=",")

# **XGBoost Model**

In [0]:
!pip install xgboost

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import roc_curve, roc_auc_score

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [0]:
from xgboost import XGBClassifier
clf1=XGBClassifier()
clf1.fit(X_train,y_train)
sfs1 = sfs(clf1,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   51.3s finished

[2020-06-07 22:51:50] Features: 1/15 -- score: 0.7019532538129466[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:  1.1min finished

[2020-06-07 22:52:54] Features: 2/15 -- score: 0.7847487299034109[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:  1.1min finished

[2020-06-07 22:54:02] Features: 3/15 -- score: 0.8268052616546264[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [0]:
feat_cols1 = list(sfs1.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols1)

Importanat faetures are present at the index:
[0, 1, 9, 10, 12, 13, 14, 15, 16, 17, 19, 21, 22, 26, 33]


Random Forest Classifier

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import roc_curve, roc_auc_score

clf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1)
sfs2 = sfs(clf2,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs2 = sfs2.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  1.2min finished

[2020-06-07 23:10:57] Features: 1/15 -- score: 0.7020852818794674[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:  1.3min finished

[2020-06-07 23:12:14] Features: 2/15 -- score: 0.7846640217482934[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:  1.3min finished

[2020-06-07 23:13:35] Features: 3/15 -- score: 0.8262418737447683[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [0]:
feat_cols2 = list(sfs2.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols2)

Importanat faetures are present at the index:
[2, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 18, 23, 24]


MLP Regressor

In [0]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import roc_curve, roc_auc_score
clf3 = MLPRegressor(random_state=1, max_iter=500)
sfs3=sfs(clf3,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs3 = sfs3.fit(X_train, y_train)

In [0]:
feat_cols3 = list(sfs3.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols3)

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, roc_auc_score
clf3a = MLPClassifier(random_state=1, max_iter=500)
sfs3a=sfs(clf3a,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs3a = sfs3a.fit(X_train, y_train)

In [0]:
feat_cols3a = list(sfs3a.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols3a)

Decision Tress Classifier

In [0]:
from sklearn.tree import DecisionTreeRegressor
clf4 = DecisionTreeRegressor(random_state=1, max_iter=500)
sfs4=sfs(clf4,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs4 = sfs4.fit(X_train, y_train)

TypeError: ignored

In [0]:
feat_cols4 = list(sfs4.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols4)

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf4a = DecisionTreeClassifier(random_state=1)
sfs4a=sfs(clf4a,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs4a = sfs4a.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    1.0s finished

[2020-06-07 23:52:33] Features: 1/15 -- score: 0.7019532538129466[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    1.2s finished

[2020-06-07 23:52:34] Features: 2/15 -- score: 0.7847268759882068[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    1.4s finished

[2020-06-07 23:52:35] Features: 3/15 -- score: 0.826152053225376[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   

In [0]:
feat_cols4a = list(sfs4a.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols4a)

Importanat faetures are present at the index:
[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 24]


Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, roc_auc_score
clf5 = LinearRegression()
sfs5=sfs(clf5,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs5 = sfs5.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    0.6s finished

[2020-06-07 23:54:08] Features: 1/15 -- score: 0.6998603537039717[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    0.7s finished

[2020-06-07 23:54:09] Features: 2/15 -- score: 0.734538268298187[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    1.2s finished

[2020-06-07 23:54:10] Features: 3/15 -- score: 0.7607760928987373[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   

In [0]:
feat_cols5 = list(sfs5.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols5)

Importanat faetures are present at the index:
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 15, 17, 18, 19, 24]


Gradient Boosting Classifier

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
clf6 = GradientBoostingClassifier(random_state=1)
sfs6=sfs(clf6,
           k_features=15,
           forward=True,
           floating=False,
           verbose=2,
           scoring='roc_auc',
           cv=5)
sfs6 = sfs6.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   46.8s finished

[2020-06-07 23:56:04] Features: 1/15 -- score: 0.7019532538129466[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:  1.1min finished

[2020-06-07 23:57:09] Features: 2/15 -- score: 0.7847534203367698[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:  1.2min finished

[2020-06-07 23:58:20] Features: 3/15 -- score: 0.8267558919541704[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [0]:
feat_cols6 = list(sfs6.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols6)

Importanat faetures are present at the index:
[1, 3, 9, 10, 12, 13, 14, 15, 16, 17, 19, 21, 22, 26, 33]


# **After feature engg...creating new dataframe with important features**

main_features_seas=['h1n1_concern','h1n1_knowledge','behavioral_avoidance','behavioral_touch_face','doctor_recc_seasonal','child_under_6_months','health_worker','opinion_h1n1_sick_from_vacc','opinion_seas_vacc_effective','opinion_seas_risk','opinion_seas_sick_from_vacc']
#train_X = X_train[main_features]
#test_X=X_test[main_features]

train_X1 = X[main_features_seas]
test_X1=test_file_new[main_features_seas]

In [0]:
main_features_h1n1=['h1n1_concern','behavioral_avoidance','behavioral_face_mask','behavioral_wash_hands','behavioral_touch_face','doctor_recc_h1n1','chronic_med_condition','child_under_6_months','health_worker','health_insurance','opinion_h1n1_vacc_effective','opinion_h1n1_risk','opinion_h1n1_sick_from_vacc','opinion_seas_vacc_effective','age_group','race']
#train_X = X_train[main_features]
#test_X=X_test[main_features]

train_X1 = X[main_features_h1n1]
test_X1=test_file_new[main_features_h1n1]

In [0]:
clf11=XGBClassifier()
clf11.fit(train_X1,y_seas)
y_pred1=clf11.predict_proba(test_X1)
#roc_auc_score(y_test,y_pred5)

In [0]:
print(y_pred1)

[[0.6399094  0.36009064]
 [0.92933154 0.07066847]
 [0.27770537 0.7222946 ]
 ...
 [0.7636051  0.2363949 ]
 [0.68522125 0.31477875]
 [0.33265233 0.66734767]]


In [0]:
np.savetxt('subm_h1n1_xgb_features_myvaladd.csv',y_pred1,delimiter=",")

# **Random Forest Model**

main_features2=['h1n1_concern','behavioral_touch_face','doctor_recc_h1n1','child_under_6_months','health_worker','health_insurance','opinion_h1n1_vacc_effective','opinion_seas_vacc_effective','opinion_seas_risk','household_children']
#train_X = X_train[main_features]
#test_X=X_test[main_features]

train_X2 = X[main_features2]
test_X2=test_file_new[main_features2]

In [0]:
clf2= RandomForestClassifier()
clf2.fit(train_X1,y_seas)
y_pred2=clf2.predict_proba(test_X1)
#roc_auc_score(y_test,y_pred5)

In [0]:
print(y_pred2)

[[0.94933333 0.05066667]
 [1.         0.        ]
 [0.15166667 0.84833333]
 ...
 [0.68341667 0.31658333]
 [0.788      0.212     ]
 [0.40261905 0.59738095]]


In [0]:
np.savetxt('subm_h1n1_randomforest_features_myvaladd.csv',y_pred2,delimiter=",")

MLP Class

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, roc_auc_score
clf3= MLPClassifier()
clf3.fit(train_X1,y_seas)
y_pred3=clf3.predict_proba(test_X1)
#roc_auc_score(y_test,y_pred5)

In [0]:
print(y_pred3)

[[0.66417762 0.33582238]
 [0.81149027 0.18850973]
 [0.52320308 0.47679692]
 ...
 [0.72959624 0.27040376]
 [0.76438039 0.23561961]
 [0.34205505 0.65794495]]


In [0]:
np.savetxt('subm_h1n1_MLPclass_features_myvaladd.csv',y_pred3,delimiter=",")

Decision Tree Classifier

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf4= DecisionTreeClassifier()
clf4.fit(train_X1,y_seas)
y_pred4=clf4.predict_proba(test_X1)

In [0]:
print(y_pred4)

[[1.         0.        ]
 [1.         0.        ]
 [0.         1.        ]
 ...
 [0.66666667 0.33333333]
 [1.         0.        ]
 [1.         0.        ]]


In [0]:
np.savetxt('subm_h1n1_decisiontree_features_myvaladd.csv',y_pred4,delimiter=",")

Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, roc_auc_score
clf5=LinearRegression()
clf5.fit(train_X1,y_seas)
y_pred5=clf5.predict(test_X1).astype(float)

In [0]:
print(y_pred5)

[0.35864604 0.30524524 0.52956854 ... 0.30520606 0.52975503 0.49207441]


In [0]:
np.savetxt('subm_h1n1_LinearReg_features_myvaladd.csv',y_pred5,delimiter=",")

Gradient Boosting Classifier

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
clf6 = GradientBoostingClassifier()
clf6.fit(train_X1,y_seas)
y_pred6=clf6.predict_proba(test_X1)

In [0]:
print(y_pred6)

[[0.64256798 0.35743202]
 [0.92994944 0.07005056]
 [0.2682842  0.7317158 ]
 ...
 [0.76374329 0.23625671]
 [0.67847321 0.32152679]
 [0.30212455 0.69787545]]


In [0]:
np.savetxt('subm_h1n1_Gradientboost_features_myvaladd.csv',y_pred5,delimiter=",")